# Script Contents
### -Import Libraries

### -Import Data

### -Aggregate Mean of Department

### -Import Data Part 2

### -Combine Data

### -Confirm and Investigate Combined Data

### -Export Combined Data

# 1. Create a new notebook for this task. Be sure to import the relevant libraries, along with your ords_prods_merge dataframe, which should include your newly derived columns from the previous Exercise.

In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [5]:
# Create a path to my main project folder
path=r"C:\Users\jacks\Instacart Basket Analysis 081424"

In [7]:
# Import latest orders_products_merged data
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_derived.pkl'))

# 2. In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [10]:
# Use the groupby () and agg() functions to find the aggregated mean of order numbers grouped by dept. ids
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# 3. Analyze the result. How do the results for the entire dataframe differ from those of the subset? 

### The results for the entire dataframe are only slightly different from the results of the subset. Some departments' average order numbers increased slighty in the new results while some averages decreased slightly. The vast majority of the averages only increased or decreased by less than 1 order number in the new results. No averages stand out as being significantly different in the new results. 

# 4. Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

In [15]:
# Create a new column, 'max_order', that will contain the maximum order number for each user
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\jacks\AppData\Local\Temp\ipykernel_6404\3743445278.py:2: FutureWarning: The provided callable <function max at 0x00000298D0AB5F80> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [17]:
# Check that the new column exists
ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,exists,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,2539329,1,1,2,8,7.0,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2539329,1,1,2,8,7.0,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
2,2539329,1,1,2,8,7.0,12427,3,0,Original Beef Jerky,23,19,4.4,both,Low-range product,Regularly busy,Regularly busy,Average orders,10
3,2539329,1,1,2,8,7.0,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both,Low-range product,Regularly busy,Regularly busy,Average orders,10
4,2539329,1,1,2,8,7.0,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both,Low-range product,Regularly busy,Regularly busy,Average orders,10


In [19]:
# Create a loyatly flag based on users' max order count
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [21]:
# Check the results of the loyalty flag by getting a value count of each customer type
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

# 5. The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [32]:
# Calculate some basic stats for the prices paid by customers for each level of customer loyalty
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean', 'min', 'max', 'median', 'std']})

prices                            
                      mean  min   max median       std
loyalty_flag                                          
Loyal customer    7.774456  1.0  25.0    7.4  4.206615
New customer      7.802351  1.0  25.0    7.4  4.267353
Regular customer  7.799316  1.0  25.0    7.4  4.255379

In [34]:
# There appears to be no significant differences. I'd also like to compare modes.
ords_prods_merge.groupby(['loyalty_flag'])['prices'].agg(pd.Series.mode)

loyalty_flag
Loyal customer      12.3
New customer        12.3
Regular customer    12.3
Name: prices, dtype: float64

### There is no significant difference in the prices of products purchased by loyal customers compared to the prices of products purchased by new and regular customers. On average, new customers purchase more expensive products than loyal and regular customers but only by a few cents. 

# 6. The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria: 
If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender. 
If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [46]:
# Create a new column, 'avg_price', that will contain the average price paid by each user
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jacks\AppData\Local\Temp\ipykernel_6404\3264815262.py:2: FutureWarning: The provided callable <function mean at 0x00000298D0AB6980> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [48]:
# Create a spending flag based on users' average price
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [50]:
# Check the results of the spending flag by getting a value count of each customer type
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     32284898
High spender      119961
Name: count, dtype: int64

### There are very few high spending customers compared to the number of low spending customers

# 7. In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [54]:
# Create a new column, 'median_days_since', that will contain the median nummber of days since each customers' prior order
ords_prods_merge['median_days_since'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\jacks\AppData\Local\Temp\ipykernel_6404\3593745194.py:2: FutureWarning: The provided callable <function median at 0x00000298D0C056C0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merge['median_days_since'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [56]:
# Create a frequency flag based on users' order frequency
ords_prods_merge.loc[ords_prods_merge['median_days_since'] > 20, 'frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_since'] <= 20) & (ords_prods_merge['median_days_since'] > 10), 'frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_since'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [58]:
# Check the results of the frequency flag by getting a value count of each customer type
ords_prods_merge['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        22790523
Regular customer          6927608
Non-frequent customer     2686728
Name: count, dtype: int64

### Most customers order frequently

In [81]:
# checking some specific columns to see if anything looks off or strange
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_since', 'frequency_flag']].head(50)

,user_id,days_since_prior_order,median_days_since,frequency_flag
0,1,7.0,20.0,Regular customer
1,1,7.0,20.0,Regular customer
2,1,7.0,20.0,Regular customer
3,1,7.0,20.0,Regular customer
4,1,7.0,20.0,Regular customer
5,1,15.0,20.0,Regular customer
6,1,15.0,20.0,Regular customer
7,1,15.0,20.0,Regular customer
8,1,15.0,20.0,Regular customer
9,1,15.0,20.0,Regular customer


In [83]:
# One last check of some column tails
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_since', 'frequency_flag']].tail(30)

,user_id,days_since_prior_order,median_days_since,frequency_flag
32404829,206209,30.0,18.0,Regular customer
32404830,206209,18.0,18.0,Regular customer
32404831,206209,18.0,18.0,Regular customer
32404832,206209,18.0,18.0,Regular customer
32404833,206209,18.0,18.0,Regular customer
32404834,206209,18.0,18.0,Regular customer
32404835,206209,18.0,18.0,Regular customer
32404836,206209,18.0,18.0,Regular customer
32404837,206209,18.0,18.0,Regular customer
32404838,206209,18.0,18.0,Regular customer


# 9. Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [86]:
# Exporting grouped dataframe as a pickle file and giving the exported file a unique name
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_grouped.pkl'))